In [2]:
import os
import pandas as pd
import numpy as np
from scipy import misc
from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
np.random.seed(31415) # for reproducibility

In [3]:
test_path = "/root/sharedfolder/test/"

In [4]:
test_img = os.listdir(test_path)

In [5]:
batch_size = 128
nb_classes = 5
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 96, 96
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (5, 5)
# convolution kernel size
kernel_size = (10, 10)

In [6]:
# convert images to numpy.ndarray
images = np.empty(shape=[len(test_img), img_rows, img_cols])
for i in range(0,len(test_img)):
    temp = test_path + test_img[i]
    img = misc.imread(temp, flatten=True,)
    img = misc.imresize(img, (img_rows, img_cols))
    images[i] = img    

In [8]:
images.shape

(263, 96, 96)

In [9]:
X_test = images.reshape(images.shape[0], 1, img_rows, img_cols)

In [10]:
X_test.shape

(263, 1, 96, 96)

In [11]:
input_shape = (1, img_rows, img_cols)

In [12]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [13]:
model.load_weights('wc31415_weights.hdf5')

In [14]:
y_pred = model.predict(X_test)

In [15]:
y_pred

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.]])

In [16]:
# 'neutral' = 0
# 'happiness' = 1
# 'fear' = 2
# 'anger' = 3
# 'sadness' = 4

In [17]:
# create list for emotions predictions
pred = [None] * len(test_img)

In [18]:
# find index of nonzero entry 
# (index number = emotion prediction)
for i in range(0,len(test_img)):
    x = np.nonzero(y_pred[i])
    pred[i] = x[0][0]    

In [19]:
kaggle = pd.DataFrame({'Image': test_img, 'Emotion': pred})

In [20]:
# convert emotions predictions from number to text
kaggle.replace(0, 'neutral', inplace=True)
kaggle.replace(1, 'happiness', inplace=True)
kaggle.replace(2, 'fear', inplace=True)
kaggle.replace(3, 'anger', inplace=True)
kaggle.replace(4, 'sadness', inplace=True)

In [21]:
kaggle.to_csv('wc31415.csv')